In [2]:
# ============================================================
# pair_generate_v10_best.py
# - raw/train.csv → train_month.csv (B안: hs4 포함, type 제외)
# - train_month → pivot (ym × item)
# - threshold sweep (0.25~0.40)로 best threshold 자동 산정
# - best threshold로 pairs_v10_thXX.csv + pairs_v10_best.csv 저장
# ============================================================

from __future__ import annotations
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
from pathlib import Path

# ============================================================
# 0. PATH
# ============================================================
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

RAW_PATH          = DATA_DIR / "raw" / "train.csv"
PROCESSED_DIR     = DATA_DIR / "processed"
TRAIN_MONTH_PATH  = PROCESSED_DIR / "train_month.csv"  # B안 결과 (팀원 코드 호환)
OUT_DIR           = PROCESSED_DIR / "v10_pairs"

OUT_DIR.mkdir(parents=True, exist_ok=True)

# 파일들
PIVOT_PATH        = OUT_DIR / "monthly_pivot_v10.csv"
THRESH_STAT_PATH  = OUT_DIR / "threshold_stats_v10.csv"
BEST_PAIR_PATH    = OUT_DIR / "pairs_v10_best.csv"


# ============================================================
# 1. raw → train_month(B안) + pivot 생성
#    - B안: (item_id, year, month, hs4, value)
# ============================================================
def build_train_month_and_pivot(
    raw_path: Path,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    입력: raw/train.csv
    출력:
      - train_month: item_id, year, month, hs4, value
      - monthly: item_id, ym, value_sum
      - pivot: index=ym, columns=item_id, values=value_sum
    """

    print(f"[LOAD RAW] {raw_path}")
    df = pd.read_csv(raw_path)

    # 기본 타입 정리
    df["year"] = df["year"].astype(int)
    df["month"] = df["month"].astype(int)
    df["hs4"] = df["hs4"].astype(str).str.zfill(4)
    df["value"] = df["value"].astype(float)

    # ym 생성
    df["ym"] = pd.to_datetime(
        df["year"].astype(str) + "-" +
        df["month"].astype(str) + "-01"
    )

    # -------------------------------
    # 1) B안 train_month (hs4 포함)
    #    - seq, weight, quantity, type 제거
    #    - 월별 value 합계
    # -------------------------------
    group_cols = ["item_id", "hs4", "year", "month"]

    train_month = (
        df.groupby(group_cols, as_index=False)["value"]
          .sum()
    )

    # 팀원 v3와 완전 호환되는 형태 유지 (train_month.csv)
    train_month.to_csv(TRAIN_MONTH_PATH, index=False)
    print(f"[SAVE] train_month(B안) → {TRAIN_MONTH_PATH}  (shape={train_month.shape})")

    # -------------------------------
    # 2) monthly (item_id × ym)
    # -------------------------------
    monthly = (
        df.groupby(["item_id", "ym"], as_index=False)["value"]
          .sum()
          .rename(columns={"value": "value_sum"})
    )

    # -------------------------------
    # 3) pivot (ym × item_id)
    # -------------------------------
    pivot = (
        monthly.pivot_table(
            index="ym",
            columns="item_id",
            values="value_sum",
            aggfunc="sum",
        )
        .sort_index()
        .fillna(0.0)
    )

    pivot.to_csv(PIVOT_PATH)
    print(f"[SAVE] monthly pivot → {PIVOT_PATH}  (shape={pivot.shape})")

    return train_month, monthly, pivot


# ============================================================
# 2. corr 기반 pair mining: base_pairs 생성
#    (once 계산 → threshold별 filter만)
# ============================================================
def safe_corr(x: np.ndarray, y: np.ndarray) -> float:
    """분산=0 예외를 처리한 피어슨 상관계수."""
    if x.std() == 0 or y.std() == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])


def mine_all_pairs(
    pivot: pd.DataFrame,
    max_lag: int = 6,
    min_nonzero: int = 8,
) -> pd.DataFrame:
    """
    팀원 v3 로직에서 threshold만 제거한 버전:
    - 모든 (leader, follower) 조합에 대해
      best_lag, max_corr 계산
    - corr_threshold는 나중에 filter에서 사용
    """

    items = pivot.columns.tolist()
    pairs: List[Dict[str, float]] = []

    print(f"[PAIR-MINE] candidate items: {len(items)}")
    values_np = pivot.values  # (T, N)

    for i, leader in enumerate(items):
        leader_series = values_np[:, i].astype(float)
        if np.count_nonzero(leader_series) < min_nonzero:
            continue

        for j, follower in enumerate(items):
            if i == j:
                continue

            follower_series = values_np[:, j].astype(float)
            if np.count_nonzero(follower_series) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            for lag in range(1, max_lag + 1):
                if len(leader_series) <= lag:
                    break

                x = leader_series[:-lag]
                y = follower_series[lag:]
                corr = safe_corr(x, y)

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            if best_lag is None:
                continue

            pairs.append(
                {
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": int(best_lag),
                    "max_corr": float(best_corr),
                }
            )

    base_pairs = pd.DataFrame(pairs)
    print(f"[PAIR-MINE] mined {len(base_pairs)} base pairs (no threshold filter yet)")
    return base_pairs


# ============================================================
# 3. threshold sweep + heuristic 점수로 best threshold 선택
# ============================================================
def evaluate_thresholds(
    base_pairs: pd.DataFrame,
    thresholds: List[float],
) -> Tuple[pd.DataFrame, float]:
    """
    thresholds 리스트에 대해:
      - pair_count
      - avg_abs_corr
      - avg_lag
      - max_lag_share
    를 계산하고, heuristic 점수로 best threshold 선택.
    """

    stats = []

    for th in thresholds:
        pairs_th = base_pairs[base_pairs["max_corr"].abs() >= th].copy()
        pair_count = len(pairs_th)

        if pair_count == 0:
            print(f"[TH={th:.3f}] no pairs, skip")
            continue

        avg_abs_corr = pairs_th["max_corr"].abs().mean()
        avg_lag = pairs_th["best_lag"].mean()

        lag_dist = pairs_th["best_lag"].value_counts(normalize=True)
        max_lag_share = float(lag_dist.max())

        stats.append(
            {
                "threshold": th,
                "pair_count": pair_count,
                "avg_abs_corr": avg_abs_corr,
                "avg_lag": avg_lag,
                "max_lag_share": max_lag_share,
            }
        )

        print(
            f"[TH={th:.3f}] pairs={pair_count}, "
            f"avg|corr|={avg_abs_corr:.4f}, avg_lag={avg_lag:.2f}, "
            f"max_lag_share={max_lag_share:.3f}"
        )

    if not stats:
        raise ValueError("모든 threshold 후보에서 pair가 생성되지 않았습니다.")

    stats_df = pd.DataFrame(stats)

    # ---------------------------
    # heuristic scoring
    #  - corr, pair_count 둘 다 normalize 후 가중합
    #  - 너무 많거나 너무 적은 pair에는 penalty
    #  - 특정 lag에 몰리면 penalty
    # ---------------------------
    pc = stats_df["pair_count"]
    ac = stats_df["avg_abs_corr"]

    pc_norm = (pc - pc.min()) / (pc.max() - pc.min() + 1e-9)
    ac_norm = (ac - ac.min()) / (ac.max() - ac.min() + 1e-9)

    raw_score = 0.6 * ac_norm + 0.4 * pc_norm
    penalty = np.zeros(len(stats_df))

    # pair 수가 너무 많거나 너무 적으면 penalty
    for idx, row in stats_df.iterrows():
        if row["pair_count"] > 5000:
            penalty[idx] += 0.3
        elif row["pair_count"] < 800:
            penalty[idx] += 0.3

        # lag 한쪽으로 심하게 몰리면 penalty
        if row["max_lag_share"] > 0.8:
            penalty[idx] += 0.2

    final_score = raw_score - penalty

    stats_df["pc_norm"] = pc_norm
    stats_df["ac_norm"] = ac_norm
    stats_df["raw_score"] = raw_score
    stats_df["penalty"] = penalty
    stats_df["final_score"] = final_score

    # best threshold
    best_idx = int(final_score.idxmax())
    best_th = float(stats_df.loc[best_idx, "threshold"])

    print("\n[THRESHOLD STATS]")
    print(stats_df.sort_values("threshold"))

    print(
        f"\n[BEST] threshold={best_th:.3f} "
        f"(final_score={stats_df.loc[best_idx, 'final_score']:.4f}, "
        f"pairs={int(stats_df.loc[best_idx, 'pair_count'])}, "
        f"avg|corr|={stats_df.loc[best_idx, 'avg_abs_corr']:.4f})"
    )

    # 저장
    stats_df.to_csv(THRESH_STAT_PATH, index=False)
    print(f"[SAVE] threshold stats → {THRESH_STAT_PATH}")

    return stats_df, best_th


# ============================================================
# 4. main: raw → train_month + pivot → base_pairs → best threshold → pairs 저장
# ============================================================
def main():
    # 1) raw → train_month(B안) + monthly + pivot
    train_month, monthly, pivot = build_train_month_and_pivot(RAW_PATH)

    # 2) 모든 pair mining (threshold 없이)
    base_pairs = mine_all_pairs(
        pivot,
        max_lag=6,
        min_nonzero=8,
    )

    # 3) threshold candidates (팀원이 쓰던 구간 그대로)
    threshold_candidates = [0.25, 0.28, 0.30, 0.32, 0.35, 0.38, 0.40]

    # 4) threshold sweep + heuristic 선택
    stats_df, best_th = evaluate_thresholds(
        base_pairs,
        threshold_candidates,
    )

    # 5) best_th에 맞는 pairs 필터링
    best_pairs = base_pairs[base_pairs["max_corr"].abs() >= best_th].copy()
    best_pairs = best_pairs.reset_index(drop=True)

    # 6) 저장
    best_th_tag = int(best_th * 100)
    best_pairs_path_tag = OUT_DIR / f"pairs_v10_th{best_th_tag:02d}.csv"

    best_pairs.to_csv(best_pairs_path_tag, index=False)
    best_pairs.to_csv(BEST_PAIR_PATH, index=False)

    print(f"[SAVE] best pairs (th={best_th:.3f}) → {best_pairs_path_tag} (rows={len(best_pairs)})")
    print(f"[SAVE] best pairs alias → {BEST_PAIR_PATH}")
    print("✅ pair_generate_v10_best DONE")


if __name__ == "__main__":
    main()


[LOAD RAW] /data/ephemeral/home/data/raw/train.csv
[SAVE] train_month(B안) → /data/ephemeral/home/data/processed/train_month.csv  (shape=(3776, 5))
[SAVE] monthly pivot → /data/ephemeral/home/data/processed/v10_pairs/monthly_pivot_v10.csv  (shape=(43, 100))
[PAIR-MINE] candidate items: 100
[PAIR-MINE] mined 8556 base pairs (no threshold filter yet)
[TH=0.250] pairs=4864, avg|corr|=0.3717, avg_lag=3.57, max_lag_share=0.195
[TH=0.280] pairs=3941, avg|corr|=0.3969, avg_lag=3.56, max_lag_share=0.193
[TH=0.300] pairs=3405, avg|corr|=0.4137, avg_lag=3.54, max_lag_share=0.188
[TH=0.320] pairs=2927, avg|corr|=0.4307, avg_lag=3.56, max_lag_share=0.193
[TH=0.350] pairs=2289, avg|corr|=0.4576, avg_lag=3.55, max_lag_share=0.187
[TH=0.380] pairs=1796, avg|corr|=0.4833, avg_lag=3.54, max_lag_share=0.180
[TH=0.400] pairs=1513, avg|corr|=0.5008, avg_lag=3.57, max_lag_share=0.187

[THRESHOLD STATS]
   threshold  pair_count  avg_abs_corr   avg_lag  max_lag_share   pc_norm  \
0       0.25        4864     